# Join and Subqueries PostgreSQL Exercises

This is part two of my working through the problem exercises on href='https://www.pgexercises.com'>pgexercises.com</a>. All exercises are using data from a hypothetical country club with information on facility usage. You can download the dataset <a href='https://www.pgexercises.com/dbfiles/clubdata.sql'>here</a>. See the 'pgexercises 1: Basic' notebook to see dataset schema. 

For this series, all problem text is just pulled from the site, followed by my solution. I'm using pandas to display the results so I can use jupyter's built in display formatting for dataframes. 

### Initial setup and helper functions:

since I use the same function through this notebook series, I have a .py file I load up. I display it here for reference. 

In [2]:
# %load pgexercises.py
#import postgres interface for python
import psycopg2
import os
import json
import pandas as pd
from IPython.display import display

#keeping output from each cell a little smaller for easy scrolling.
pd.set_option('display.max_rows',14)

with open('config.json') as f:
    conf = json.load(f) 

postgres_string = "dbname='{database}' user='{user}' host='{host}' password='{passw}'".format(**conf)

relative_data_path = '../../Data/SQL'
database_filename = 'clubdata.sql'

try:
    conn = psycopg2.connect("dbname='exercises' user='james' host='localhost' password='fakepassword'")
except:
    print("couldn't connect")
cursor = conn.cursor()

#helper function for printing. For better jupyter formatting, I'm just using panda's built in read function. 
def print_output(query):
    global conn
    display(pd.read_sql_query(query, conn))


How can you produce a list of the start times for bookings by members named 'David Farrell'?

In [4]:
q = """
    SELECT bks.starttime 
        FROM 
            cd.bookings bks 
            INNER JOIN cd.members mems
                ON mems.memid = bks.memid
        WHERE 
            mems.firstname = 'David' 
            AND mems.surname = 'Farrell';"""

print_output(q)

,starttime
0,2012-09-18 09:00:00
1,2012-09-18 17:30:00
2,2012-09-18 13:30:00
3,2012-09-18 20:00:00
4,2012-09-19 09:30:00
5,2012-09-19 15:00:00
6,2012-09-19 12:00:00
...,...
27,2012-09-28 13:00:00
28,2012-09-29 16:00:00


How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.

In [6]:
q = """
    SELECT bks.starttime, fac.name
    FROM cd.bookings bks 
    INNER JOIN cd.facilities fac
        ON bks.facid = fac.facid
    WHERE fac.facid in (0,1)
        AND bks.starttime >= '2012-09-21'
        AND bks.starttime < '2012-09-22';"""

print_output(q)

,starttime,name
0,2012-09-21 08:00:00,Tennis Court 1
1,2012-09-21 08:00:00,Tennis Court 2
2,2012-09-21 09:30:00,Tennis Court 1
3,2012-09-21 10:00:00,Tennis Court 2
4,2012-09-21 11:30:00,Tennis Court 2
5,2012-09-21 12:00:00,Tennis Court 1
6,2012-09-21 13:30:00,Tennis Court 1
7,2012-09-21 14:00:00,Tennis Court 2
8,2012-09-21 15:30:00,Tennis Court 1
9,2012-09-21 16:00:00,Tennis Court 2


How can you output a list of all members who have recommended another member? Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).

In [7]:
q = """
    SELECT mem.firstname, mem.surname
    FROM cd.members mem 
    WHERE mem.memid IN 
        (SELECT DISTINCT mem.recommendedby
            FROM cd.members mem)
    ORDER BY (surname, firstname);"""
print_output(q)

,firstname,surname
0,Florence,Bader
1,Timothy,Baker
2,Gerald,Butters
3,Jemima,Farrell
4,Matthew,Genting
5,David,Jones
6,Janice,Joplette
7,Millicent,Purview
8,Tim,Rownam
9,Darren,Smith


How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).

In [8]:
q = """
    SELECT mems.firstname AS memfname, mems.surname AS memsname, 
        recs.firstname AS recfname, recs.surname AS recsname
    FROM
        cd.members mems
        LEFT OUTER JOIN cd.members recsq = """
    SELECT mems.firstname AS memfname, mems.surname AS memsname, 
        recs.firstname AS recfname, recs.surname AS recsname
    FROM
        cd.members mems
        LEFT OUTER JOIN cd.members recs
        ON recs.memid = mems.recommendedby
    ORDER BY memsname, memfname;"""
print_output(q)
        ON recs.memid = mems.recommendedby
    ORDER BY memsname, memfname;"""
print_output(q)

,memfname,memsname,recfname,recsname
0,Florence,Bader,Ponder,Stibbons
1,Anne,Baker,Ponder,Stibbons
2,Timothy,Baker,Jemima,Farrell
3,Tim,Boothe,Tim,Rownam
4,Gerald,Butters,Darren,Smith
5,Joan,Coplin,Timothy,Baker
6,Erica,Crumpet,Tracy,Smith
...,...,...,...,...
24,Darren,Smith,None,None
25,Jack,Smith,Darren,Smith


How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [ ]:
q = """
    SELECT mems.firstname AS memfname, mems.surname AS memsname, 
        recs.firstname AS recfname, recs.surname AS recsname
    FROM
        cd.members mems
        LEFT OUTER JOIN cd.members recs
        ON recs.memid = mems.recommendedby
    ORDER BY memsname, memfname;"""
print_output(q)

How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [9]:
q = """SELECT DISTINCT mem.firstname || ' ' || mem.surname AS member, fac.name as facility
    FROM cd.members mem
    INNER JOIN cd.bookings bks
        ON bks.memid = mem.memid
    INNER JOIN cd.facilities fac
        ON bks.facid = fac.facid
    WHERE fac.facid IN (0,1)
    ORDER BY member;"""
print_output(q)

,member,facility
0,Anne Baker,Tennis Court 2
1,Anne Baker,Tennis Court 1
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
...,...,...
39,Tim Boothe,Tennis Court 2
40,Timothy Baker,Tennis Court 1


How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [12]:
q = """
    SELECT member, facility, cost FROM (
        SELECT mem.firstname || ' ' || mem.surname AS member,
            fac.name AS facility,
            CASE
                WHEN mem.memid = 0 THEN bks.slots * fac.guestcost
                ELSE bks.slots * fac.membercost
            END AS cost
        FROM cd.members mem
        INNER JOIN cd.bookings bks
            ON bks.memid = mem.memid
        INNER JOIN cd.facilities fac
            ON bks.facid = fac.facid
        WHERE bks.starttime >= '2012-09-14' AND
            bks.starttime < '2012-09-15') AS bookings
    WHERE cost > 30
    ORDER BY cost DESC;"""
print_output(q)

,member,facility,cost
0,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Massage Room 1,160.0
2,GUEST GUEST,Massage Room 1,160.0
3,GUEST GUEST,Massage Room 1,160.0
4,GUEST GUEST,Tennis Court 2,150.0
5,Jemima Farrell,Massage Room 1,140.0
6,GUEST GUEST,Tennis Court 1,75.0
...,...,...,...
11,GUEST GUEST,Squash Court,70.0
12,Jemima Farrell,Massage Room 1,70.0


How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.

In [11]:
q = """
    SELECT DISTINCT mem.firstname || ' ' || mem.surname AS member,
    (SELECT recs.firstname || ' ' || recs.surname AS recommender
        FROM cd.members recs
        WHERE recs.memid = mem.recommendedby)
    FROM cd.members mem
    ORDER BY member;"""
print_output(q)

,member,recommender
0,Anna Mackenzie,Darren Smith
1,Anne Baker,Ponder Stibbons
2,Burton Tracy,None
3,Charles Owen,Darren Smith
4,Darren Smith,None
5,David Farrell,None
6,David Jones,Janice Joplette
...,...,...
23,Nancy Dare,Janice Joplette
24,Ponder Stibbons,Burton Tracy
